In [1]:
import dotenv; dotenv.load_dotenv()

True

In [2]:
import datetime

class ChatFunction:
    # ドキュメントでは optional だが、未指定の場合、このようにしないとエラーになる @ 23/6/16
    _no_parameters = {'type': 'object', 'properties': {}}

    def __call__(self, description, **_prop_descriptions):
        if not isinstance(description, str):
            raise ValueError

        def decorate(function):
            self.__dict__[function.__name__] = (
                function,
                description,
                self._no_parameters  # FIXME typing で抽出すればいい
            )

        return decorate

    @property
    def defs(self):
        return [
            {'name': name, 'description': description, 'parameters': parameters}
            for name, (_, description, parameters) in vars(self).items()
        ]

    def __str__(self):
        return '\n'.join(
            f'- {func_name} {func_info}'
            for func_name, func_info in vars(chat_function).items()
        )

chat_function = ChatFunction()

@chat_function('現在日時を表示します')
def now():
    return datetime.datetime.now().isoformat()

@chat_function('サーバーをシャットダウンします。')
def shutdownn():
    print('called `shutdown -h now`  # is dummy')

In [3]:
print(chat_function)

- now (<function now at 0x00000196B9A16830>, '現在日時を表示します', {'type': 'object', 'properties': {}})
- shutdownn (<function shutdownn at 0x00000196B9A168C0>, 'サーバーをシャットダウンします。', {'type': 'object', 'properties': {}})


## GPT-3.5

In [4]:
import json

try:
    import openai
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install openai
    import openai

model = 'gpt-3.5-turbo-0613' or 'gpt-4-0613'

messages = [
    {
        'role': 'system',
        'content': """
あなたはシステム管理者です。
サーバーのログを確認し、最後のログエントリから 1 時間以上経過していれば、システムをシャットダウンして下さい。
""".strip()
    },

    {
        'role': 'user',
        'content': f"""
最新のログ内容は、次の通り:

```log
[2023-06-16 10:00] Oshinko joined the system.
[2023-06-16 10:20] Oshinko left the system.
```

現在日時: {datetime.datetime.now().isoformat()}
""".strip()
    }
]

print('送信するメッセージ (文脈):')
print(json.dumps(messages, indent=2, ensure_ascii=False), end='\n' * 2)

resp = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=chat_function.defs
)

print('受信したメッセージ:')
print(json.dumps(resp, indent=2, ensure_ascii=False))

assistant_message = resp['choices'][0]['message']
messages.append(assistant_message)

送信するメッセージ (文脈):
[
  {
    "role": "system",
    "content": "あなたはシステム管理者です。\nサーバーのログを確認し、最後のログエントリから 1 時間以上経過していれば、システムをシャットダウンして下さい。"
  },
  {
    "role": "user",
    "content": "最新のログ内容は、次の通り:\n\n```log\n[2023-06-16 10:00] Oshinko joined the system.\n[2023-06-16 10:20] Oshinko left the system.\n```\n\n現在日時: 2023-06-16T16:32:09.578363"
  }
]

受信したメッセージ:
{
  "id": "chatcmpl-7RyEEKLr2kNUAe9NudtVoPbvxM6Gi",
  "object": "chat.completion",
  "created": 1686900730,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "now",
          "arguments": "{}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 206,
    "completion_tokens": 6,
    "total_tokens": 212
  }
}


In [5]:
messages_ = messages.copy()

if 'function_call' in assistant_message:
    function_call = assistant_message['function_call']

    messages_.append({
        'role': 'function',
        'name': function_call['name'],
        'content': json.dumps(getattr(chat_function, function_call['name'])[0]())
    })

    print('送信するメッセージ (文脈):')
    print(json.dumps(messages, indent=2, ensure_ascii=False), end='\n' * 2)

    resp = openai.ChatCompletion.create(
        model=model,
        messages=messages_,
        functions=chat_function.defs
    )

    print('受信したメッセージ:')
    print(json.dumps(resp, indent=2, ensure_ascii=False))

送信するメッセージ (文脈):
[
  {
    "role": "system",
    "content": "あなたはシステム管理者です。\nサーバーのログを確認し、最後のログエントリから 1 時間以上経過していれば、システムをシャットダウンして下さい。"
  },
  {
    "role": "user",
    "content": "最新のログ内容は、次の通り:\n\n```log\n[2023-06-16 10:00] Oshinko joined the system.\n[2023-06-16 10:20] Oshinko left the system.\n```\n\n現在日時: 2023-06-16T16:32:09.578363"
  },
  {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "now",
      "arguments": "{}"
    }
  }
]

受信したメッセージ:
{
  "id": "chatcmpl-7RyEFws38Kb54IrnSoiBhdfFkmv8M",
  "object": "chat.completion",
  "created": 1686900731,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "最後のログエントリから現在までの経過時間は約6時間12分です。したがって、システムをシャットダウンする必要はありません。"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 236,
    "completion_tokens": 53,
    "total_tokens": 289
  }
}


## GPT-4

In [6]:
model = 'gpt-4-0613'

print('送信するメッセージ (文脈):')
print(json.dumps(messages, indent=2, ensure_ascii=False), end='\n' * 2)

resp = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=chat_function.defs
)

print('受信したメッセージ:')
print(json.dumps(resp, indent=2, ensure_ascii=False))

assistant_message = resp['choices'][0]['message']
messages.append(assistant_message)

送信するメッセージ (文脈):
[
  {
    "role": "system",
    "content": "あなたはシステム管理者です。\nサーバーのログを確認し、最後のログエントリから 1 時間以上経過していれば、システムをシャットダウンして下さい。"
  },
  {
    "role": "user",
    "content": "最新のログ内容は、次の通り:\n\n```log\n[2023-06-16 10:00] Oshinko joined the system.\n[2023-06-16 10:20] Oshinko left the system.\n```\n\n現在日時: 2023-06-16T16:32:09.578363"
  },
  {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "now",
      "arguments": "{}"
    }
  }
]

受信したメッセージ:
{
  "id": "chatcmpl-7RyEHuXcDHMDLlYWU5z7tnCqHj6l5",
  "object": "chat.completion",
  "created": 1686900733,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "現在日時は 2023-06-16T16:50:00.000000 です。最後のログエントリが 2023-06-16 10:20 であるため、より1時間以上が経過しています。したがって、システムをシャットダウンします。",
        "function_call": {
          "name": "shutdownn",
          "arguments": "{}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage"

In [7]:
messages_ = messages.copy()

if 'function_call' in assistant_message:
    function_call = assistant_message['function_call']

    messages_.append({
        'role': 'function',
        'name': function_call['name'],
        'content': json.dumps(getattr(chat_function, function_call['name'])[0]())
    })

    print('送信するメッセージ (文脈):')
    print(json.dumps(messages, indent=2, ensure_ascii=False), end='\n' * 2)

    resp = openai.ChatCompletion.create(
        model=model,
        messages=messages_,
        functions=chat_function.defs
    )

    print('受信したメッセージ:')
    print(json.dumps(resp, indent=2, ensure_ascii=False))

called `shutdown -h now`  # is dummy
送信するメッセージ (文脈):
[
  {
    "role": "system",
    "content": "あなたはシステム管理者です。\nサーバーのログを確認し、最後のログエントリから 1 時間以上経過していれば、システムをシャットダウンして下さい。"
  },
  {
    "role": "user",
    "content": "最新のログ内容は、次の通り:\n\n```log\n[2023-06-16 10:00] Oshinko joined the system.\n[2023-06-16 10:20] Oshinko left the system.\n```\n\n現在日時: 2023-06-16T16:32:09.578363"
  },
  {
    "role": "assistant",
    "content": null,
    "function_call": {
      "name": "now",
      "arguments": "{}"
    }
  },
  {
    "role": "assistant",
    "content": "現在日時は 2023-06-16T16:50:00.000000 です。最後のログエントリが 2023-06-16 10:20 であるため、より1時間以上が経過しています。したがって、システムをシャットダウンします。",
    "function_call": {
      "name": "shutdownn",
      "arguments": "{}"
    }
  }
]

受信したメッセージ:
{
  "id": "chatcmpl-7RyEQTrCNBcL9PDVgZiBKkHGpudLw",
  "object": "chat.completion",
  "created": 1686900742,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content"